In [1]:
# %%
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import os 
from pathlib import Path
import sys
from tqdm import tqdm
import pycwt as wavelet
from pycwt import wct_significance
from pycwt import helpers
import matplotlib.dates as mdates
import scipy
from scipy.signal import detrend
from matplotlib.ticker import LogLocator, FormatStrFormatter

# Load dengue case data
global_dengue_national = pd.read_csv("/home/patwuch/Documents/projects/Chuang-Lab-TMU/dengue-infection-module/work/global/National_extract_V1_3.csv")
global_dengue_temporal = pd.read_csv("/home/patwuch/Documents/projects/Chuang-Lab-TMU/dengue-infection-module/work/global/Temporal_extract_V1_3.csv")
global_dengue_spatial = pd.read_csv("/home/patwuch/Documents/projects/Chuang-Lab-TMU/dengue-infection-module/work/global/Spatial_extract_V1_3.csv")

print(len(global_dengue_national))
print(len(global_dengue_temporal))
print(len(global_dengue_spatial))



/tmp/ipykernel_110577/2639077365.py:23: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  global_dengue_temporal = pd.read_csv("/home/patwuch/Documents/projects/Chuang-Lab-TMU/dengue-infection-module/work/global/Temporal_extract_V1_3.csv")
/tmp/ipykernel_110577/2639077365.py:24: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  global_dengue_spatial = pd.read_csv("/home/patwuch/Documents/projects/Chuang-Lab-TMU/dengue-infection-module/work/global/Spatial_extract_V1_3.csv")


29873
2835022
2821791


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

RES_PRIORITY = ["Week", "Month", "Year"]  # higher resolution first
RES_COLOR_MAP = {"Week": "blue", "Month": "green", "Year": "orange"}

def plot_all_countries(df, output_dir, start_date="2000-01-01"):
    df = df.copy()
    start_date = pd.to_datetime(start_date)

    # Ensure date columns are datetime
    for col in ["calendar_start_date", "calendar_end_date"]:
        df[col] = pd.to_datetime(df[col], errors="coerce")

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    countries = df["full_name"].dropna().unique()

    for country in countries:
        country_df = df[df["full_name"] == country].copy()
        country_df = country_df.dropna(subset=["calendar_start_date", "calendar_end_date"])
        if country_df.empty or country_df["calendar_end_date"].max() < start_date:
            continue

        # Clip start date
        country_df = country_df[country_df["calendar_end_date"] >= start_date]

        # Expand to daily pseudo-cases with T_res
        country_df["start_clipped"] = country_df["calendar_start_date"].apply(lambda x: max(x, start_date))
        country_df["date"] = country_df.apply(
            lambda r: pd.date_range(r["start_clipped"], r["calendar_end_date"], freq="D")
            if r["calendar_end_date"] >= r["start_clipped"] else pd.DatetimeIndex([]),
            axis=1
        )
        country_df["n_days"] = country_df["date"].str.len()
        country_df["daily_cases"] = country_df["dengue_total"] / country_df["n_days"]

        # Explode
        daily_df = country_df[["date", "daily_cases", "T_res"]].explode("date")
        if daily_df.empty:
            continue

        # Normalize T_res
        daily_df["T_res"] = daily_df["T_res"].astype(str).str.lower().str.strip()

        # -------------------------
        # For each date, pick highest resolution available
        # -------------------------
        daily_df["res_priority"] = daily_df["T_res"].apply(lambda x: RES_PRIORITY.index(x) if x in RES_PRIORITY else len(RES_PRIORITY))
        daily_df = daily_df.sort_values(["date", "res_priority"])
        daily_df = daily_df.drop_duplicates("date", keep="first")
        daily_df = daily_df.sort_values("date")  # final daily_df is continuous

        # Check for overlapping resolutions in the raw data
        raw_overlaps = daily_df.groupby("date")["T_res"].nunique()
        overlap_dates = raw_overlaps[raw_overlaps > 1].index
        if len(overlap_dates) > 0:
            print(f"WARNING: {len(overlap_dates)} dates had multiple resolutions for {country}. First 10: {list(overlap_dates[:10])}")

        # Compute cumulative cases
        daily_df["cumulative_cases"] = daily_df["daily_cases"].cumsum()

        # -------------------------
        # Helper to plot continuous line segments colored by resolution
        # -------------------------
        def plot_colored_line(ax, x, y, res_series):
            start_idx = 0
            while start_idx < len(x):
                current_res = res_series.iloc[start_idx]
                # find contiguous segment with same resolution
                end_idx = start_idx + 1
                while end_idx < len(x) and res_series.iloc[end_idx] == current_res:
                    end_idx += 1
                ax.plot(x[start_idx:end_idx], y[start_idx:end_idx],
                        color=RES_COLOR_MAP.get(current_res, "black"),
                        linewidth=1.5,
                        alpha=0.9 if current_res != "year" else 0.6)
                start_idx = end_idx

        # -------------------------
        # Plot daily + cumulative in subplots
        # -------------------------
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

        plot_colored_line(ax1, daily_df["date"], daily_df["daily_cases"], daily_df["T_res"])
        ax1.set_ylabel("Daily Cases")
        ax1.set_title(f"Dengue Cases in {country} (2000–present)")

        plot_colored_line(ax2, daily_df["date"], daily_df["cumulative_cases"], daily_df["T_res"])
        ax2.set_ylabel("Cumulative Cases")
        ax2.set_xlabel("Date")

        # Build a shared legend
        handles = [plt.Line2D([0], [0], color=color, lw=2, label=res.capitalize())
                   for res, color in RES_COLOR_MAP.items() if res in daily_df["T_res"].values]
        if handles:
            fig.legend(handles=handles, title="Time Resolution", loc="upper center", ncol=len(handles))

        fig.tight_layout(rect=[0, 0, 1, 0.95])
        fig.savefig(output_dir / f"{country}_dengue_plot.png", dpi=300)
        plt.close(fig)


In [44]:
plot_all_countries(
    df=global_dengue_national,
    output_dir="figures/dengue_national"
)


In [4]:
global_dengue_spatial

,adm_0_name,adm_1_name,adm_2_name,full_name,ISO_A0,FAO_GAUL_code,RNE_iso_code,IBGE_code,calendar_start_date,calendar_end_date,Year,dengue_total,case_definition_standardised,S_res,T_res,UUID
0,AFGHANISTAN,NaN,NaN,AFGHANISTAN,AFG,1011446,AFG,NaN,2021-09-05,2021-09-11,2021,18.0,Suspected,Admin0,Week,WHOEMRO-ALL-2021-Y01-05
1,AFGHANISTAN,NaN,NaN,AFGHANISTAN,AFG,1011446,AFG,NaN,2021-09-12,2021-09-18,2021,24.0,Suspected,Admin0,Week,WHOEMRO-ALL-2021-Y01-05
2,AFGHANISTAN,NaN,NaN,AFGHANISTAN,AFG,1011446,AFG,NaN,2021-09-19,2021-09-25,2021,9.0,Suspected,Admin0,Week,WHOEMRO-ALL-2021-Y01-05
3,AFGHANISTAN,NaN,NaN,AFGHANISTAN,AFG,1011446,AFG,NaN,2021-09-26,2021-10-02,2021,104.0,Suspected,Admin0,Week,WHOEMRO-ALL-2021-Y01-05
4,AFGHANISTAN,NaN,NaN,AFGHANISTAN,AFG,1011446,AFG,NaN,2021-10-03,2021-10-09,2021,98.0,Suspected,Admin0,Week,WHOEMRO-ALL-2021-Y01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821786,YEMEN,NaN,NaN,YEMEN,YEM,269,YEM,NaN,2024-06-23,2024-06-29,2024,139.0,Suspected and confirmed,Admin0,Week,LITERATURE-YEM-20202024-Y01-00
2821787,YEMEN,NaN,NaN,YEMEN,YEM,269,YEM,NaN,2024-06-30,2024-07-06,2024,124.0,Suspected and confirmed,Admin0,Week,LITERATURE-YEM-20202024-Y01-00
2821788,YEMEN,NaN,NaN,YEMEN,YEM,269,YEM,NaN,2024-07-07,2024-07-13,2024,121.0,Suspected and confirmed,Admin0,Week,LITERATURE-YEM-20202024-Y01-00
2821789,YEMEN,NaN,NaN,YEMEN,YEM,269,YEM,NaN,2024-07-14,2024-07-20,2024,121.0,Suspected and confirmed,Admin0,Week,LITERATURE-YEM-20202024-Y01-00


In [16]:
global_dengue_national['S_res'].value_counts(), global_dengue_national['T_res'].value_counts()

(S_res
 Admin0    29873
 Name: count, dtype: int64,
 T_res
 Week     23248
 Year      3495
 Month     3130
 Name: count, dtype: int64)

In [18]:
global_dengue_spatial['T_res'].value_counts(), global_dengue_spatial['S_res'].value_counts()

(T_res
 Week     2238858
 Month     575981
 Year        6952
 Name: count, dtype: int64,
 S_res
 Admin2    2642778
 Admin1     142245
 Admin0      36768
 Name: count, dtype: int64)

In [19]:
global_dengue_temporal['T_res'].value_counts(), global_dengue_temporal['S_res'].value_counts()

(T_res
 Week     2249270
 Month     578429
 Year        7323
 Name: count, dtype: int64,
 S_res
 Admin2    2642777
 Admin1     151794
 Admin0      40451
 Name: count, dtype: int64)

In [27]:
global_dengue_temporal['adm_1_name'].unique()

array([nan, 'CORRIENTES', 'FORMOSA', 'SANTA FE', 'MISIONES', 'CHACO',
       'SALTA', 'SANTIAGO DEL ESTERO', 'BUENOS AIRES', 'JUJUY', 'CORDOBA',
       'TUCUMAN', 'ENTRE RIOS', 'LA RIOJA', 'MENDOZA', 'NEUQUEN',
       'CATAMARCA', 'SAN LUIS', 'RIO NEGRO', 'TIERRA DEL FUEGO',
       'LA PAMPA', 'SAN JUAN', 'SANTA CRUZ', 'CHUBUT', 'ACT', 'NSW', 'NT',
       'QLD', 'SA', 'TAS', 'VIC', 'WA', 'TRASHIYANGTSE',
       'WANGDUEPHODRANG', 'ZHEMGANG', 'MONGAR', 'PARO', 'PEMAGATSEL',
       'PUNAKHA', 'SAMTSE', 'SARPANG', 'SAMDRUPJONGKHAR', 'BUMTHANG',
       'CHUKHA', 'DAGANA', 'GASA', 'HAA', 'LHUNTSE', 'THIMPU', 'TRONGSA',
       'TSIRANG', 'TRASHIGANG', 'CHUQUISACA', 'LA PAZ', 'COCHABAMBA',
       'ORURO', 'POTOSI', 'TARIJA', 'BENI', 'PANDO', 'ALAGOAS', 'AMAPA',
       'ACRE', 'GOIAS', 'MINAS GERAIS', 'PARA', 'BAHIA', 'PERNAMBUCO',
       'MARANHAO', 'CEARA', 'RIO GRANDE DO NORTE', 'PIAUI', 'SAO PAULO',
       'PARANA', 'ESPIRITO SANTO', 'MATO GROSSO', 'PARAIBA',
       'MATO GROSSO DO SUL', '